In [2]:
import pandas as pd
df=pd.read_csv('./HR.csv')
df

satisfaction_level  last_evaluation  number_project  \
0                    0.38             0.53               2   
1                    0.80             0.86               5   
2                    0.11             0.88               7   
3                    0.72             0.87               5   
4                    0.37             0.52               2   
...                   ...              ...             ...   
14996                0.11             0.96               6   
14997                0.37             0.52               2   
14998                 NaN             0.52               2   
14999                 NaN        999999.00               2   
15000                0.70             0.40               2   

       average_monthly_hours  time_spend_company  Work_accident  left  \
0                        157                   3              0     1   
1                        262                   6              0     1   
2                        272                   4              0     1   
3                        223                   5              0     1   
4                        159                   3              0     1   
...                      ...                 ...            ...   ...   
14996                    280                   4              0     1   
14997                    158                   3              0     1   
14998                    158                   3              0     1   
14999                    158                   3              0     1   
15000                    158                   2              0     1   

       promotion_last_5years department  salary  
0                          0      sales     low  
1                          0      sales  medium  
2                          0      sales  medium  
3                          0      sales     low  
4                          0      sales     low  
...                      ...        ...     ...  
14996                      0    support     low  
14997                      0    support     low  
14998                      0    support     low  
14999                      0       sale     low  
15000                      0       sale     nme  

[15001 rows x 10 columns]

In [3]:
 

from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.decomposition import PCA
 
def hr_preprocessing(sl=False,le=False,npr=False,amh=False,tsc=False,wa=False,pl5=False,dp=False,slr=False,lower_id=False,ld_n=1):
    df=pd.read_csv('./Data01.')
    #1 清洗数据，根据探索性数据分析出的结果来去掉空值
    df=df.dropna(subset=['satisfaction_level','last_evaluation'])
    df=df[df['satisfaction_level']<=1][df['salary']!='nme']
 
    #2 得到标注
    label=df['left']
    df=df.drop('left',axis=1)
 
    #3 特征选择（因为本文的特征本来就不多，暂时不做）
 
    # 4 特征处理（归一化，标准化，降维）
    scaler_lst=[sl,le,npr,amh,tsc,wa,pl5]
    column_lst = ["satisfaction_level", "last_evaluation", "number_project",
                  "average_monthly_hours", "time_spend_company", "Work_accident",
                  "promotion_last_5years"]
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
        else:
            df[column_lst[i]]=StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
 
    #针对离散值的处理
    scaler_lst=[slr,dp]
    column_lst=['salary','department']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            if column_lst[i]=='salary':
                df[column_lst[i]]=[map_salary(s)for s in df['salary'].values]
            else:
                df[column_lst[i]]=LabelEncoder().fit_transform(df[column_lst[i]])
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
        else:
            df=pd.get_dummies(df,columns=[column_lst[i]])
    if lower_id:
        return PCA(n_components=ld_n).fit_transform(df.values),label
    return df,label
 
d=dict([('low',0),('medium',1),('high',2)])
def map_salary(s):
    return d.get(s,0)
 
#建模
def hr_modeling_nn(features,label):
    from sklearn.model_selection import train_test_split
    #sklearn中没有可以一次性将数据划分成验证集、训练集、测试集的包，所以需要进行两步划分
    f_v=features.values
    l_v=label.values
    X_tt,X_validation,Y_tt,Y_validation=train_test_split(f_v,l_v,test_size=0.2)
    X_train,X_test,Y_train,Y_test=train_test_split(X_tt,Y_tt,test_size=0.25)
    #print(len(X_train),len(X_test),len(X_validation))
 
    #分类
    from sklearn.metrics import accuracy_score,recall_score,f1_score
    from sklearn.neighbors import NearestNeighbors,KNeighborsClassifier
    from sklearn.naive_bayes import GaussianNB,BernoulliNB
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.svm import SVC
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import GradientBoostingClassifier
 
    models=[]
    models.append(('KNN',KNeighborsClassifier(n_neighbors=3)))
    models.append(('GaussianNB',GaussianNB()))
    models.append(('BernoulliNB',BernoulliNB()))
    models.append(('DecisionTreeGini',DecisionTreeClassifier()))
    models.append(('DecisionTreeEntropy',DecisionTreeClassifier(criterion='entropy')))
    models.append(('SVM:',SVC(C=1000)))
    models.append(('OriginalRandomForest',RandomForestClassifier()))
    models.append(('RandomForest',RandomForestClassifier(n_estimators=11,max_features=None)))
    models.append(('Adaboost',AdaBoostClassifier(n_estimators=100)))
    models.append(('LogisticRegression',LogisticRegression(C=1000,tol=1e-10,solver='sag',max_iter=10000)))
    models.append(('GBDT',GradientBoostingClassifier(max_depth=6,n_estimators=100)))
 
 
    for clf_name ,clf in models:
        clf.fit(X_train,Y_train)
        xy_list=[(X_train,Y_train),(X_validation,Y_validation),(X_test,Y_test)]
        for i in range(len(xy_list)):
            X_part=xy_list[i][0]
            Y_part=xy_list[i][1]
            Y_pred=clf.predict(X_part)
            print(i)
            print(clf_name,'-ACC',accuracy_score(Y_part,Y_pred))
            print(clf_name,'-REC:',recall_score(Y_part,Y_pred))
            print(clf_name,'-F1:',f1_score(Y_pred,Y_part))
 
#回归
def regr_t(features,label):
    print('X',features)
    print('Y',label)
    from sklearn.linear_model import LinearRegression,Ridge,Lasso
    regr=Ridge(alpha=1)
    regr.fit(features.values,label.values)
    Y_pred=regr.predict(features.values)
    print('Coef:',regr.coef_)
    from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
    print('MSE:',mean_squared_error(label.values,Y_pred))
    print('MAE:',mean_absolute_error(label.values,Y_pred))
    print('R2:',r2_score(label.values,Y_pred))
 
 
def main():
    features,label=hr_preprocessing()
    hr_modeling_nn(features,label)
    regr_t(features[['number_project','average_monthly_hours']],features['last_evaluation'])
if __name__=='__main__':
    main()
 
 
 
 

0
KNN -ACC 0.9754389864414315
KNN -REC: 0.9621421975992613
KNN -F1: 0.9496468443836864
1
KNN -ACC 0.9536666666666667
KNN -REC: 0.9267241379310345
KNN -F1: 0.9027291812456262
2
KNN -ACC 0.9583333333333334
KNN -REC: 0.9265536723163842
KNN -F1: 0.9130132219902576
0
GaussianNB -ACC 0.7929539897755057
GaussianNB -REC: 0.7576177285318559
GaussianNB -F1: 0.6379008746355684
1
GaussianNB -ACC 0.7916666666666666
GaussianNB -REC: 0.728448275862069
GaussianNB -F1: 0.6186699206833434
2
GaussianNB -ACC 0.792
GaussianNB -REC: 0.769774011299435
GaussianNB -F1: 0.6359393232205367
0
BernoulliNB -ACC 0.8396310291175817
BernoulliNB -REC: 0.46860572483841184
BernoulliNB -F1: 0.5845090699683271
1
BernoulliNB -ACC 0.842
BernoulliNB -REC: 0.46695402298850575
BernoulliNB -F1: 0.5782918149466192
2
BernoulliNB -ACC 0.8473333333333334
BernoulliNB -REC: 0.4689265536723164
BernoulliNB -F1: 0.5918003565062389
0
DecisionTreeGini -ACC 1.0
DecisionTreeGini -REC: 1.0
DecisionTreeGini -F1: 1.0
1
DecisionTreeGini -ACC 0.9

/Users/kefengyinyinyin....../anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0
SVM: -ACC 0.9628806401422538
SVM: -REC: 0.91735918744229
SVM: -F1: 0.9224698235840297
1
SVM: -ACC 0.9583333333333334
SVM: -REC: 0.9137931034482759
SVM: -F1: 0.910522548317824
2
SVM: -ACC 0.958
SVM: -REC: 0.9110169491525424
SVM: -F1: 0.9110169491525424


/Users/kefengyinyinyin....../anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0
OriginalRandomForest -ACC 0.9982218270726828
OriginalRandomForest -REC: 0.9935364727608494
OriginalRandomForest -F1: 0.9962962962962963
1
OriginalRandomForest -ACC 0.9886666666666667
OriginalRandomForest -REC: 0.9525862068965517
OriginalRandomForest -F1: 0.975
2
OriginalRandomForest -ACC 0.988
OriginalRandomForest -REC: 0.9576271186440678
OriginalRandomForest -F1: 0.9741379310344828
0
RandomForest -ACC 0.9981106912647255
RandomForest -REC: 0.9935364727608494
RandomForest -F1: 0.9960657255264984
1
RandomForest -ACC 0.987
RandomForest -REC: 0.9597701149425287
RandomForest -F1: 0.9716363636363636
2
RandomForest -ACC 0.985
RandomForest -REC: 0.961864406779661
RandomForest -F1: 0.9680170575692963
0
Adaboost -ACC 0.9624360969104245
Adaboost -REC: 0.9145891043397969
Adaboost -F1: 0.9213953488372093
1
Adaboost -ACC 0.965
Adaboost -REC: 0.9367816091954023
Adaboost -F1: 0.9254790631653654
2
Adaboost -ACC 0.9603333333333334
Adaboost -REC: 0.9209039548022598
Adaboost -F1: 0.9163738580463808
0
Lo